In [25]:
from pandas import read_csv
import pandas as pd
import autogluon as ag
from autogluon import TabularPrediction as task
import joblib

In [26]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [27]:
#Train
#Price features
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Category count
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Category sum
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Category min and max
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Category average sum
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Merge all the features
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [28]:
train_size = 500
train_data = train.head(train_size)

In [29]:
import numpy as np
np.bincount(train_data['bins'])

array([122, 119, 120, 139], dtype=int64)

In [31]:
import time
start_time = time.time()

In [32]:
predictor = task.fit(train_data=train_data, label='bins', problem_type="multiclass", auto_stack=True, verbosity=4)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20201027_140459\
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20201027_140459\
Train Data Rows:    500
Train Data Columns: 1015
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 500 data points with 864 features
Original Features:
	int features: 1
	float features: 863
	object features: 0
Generated Features:
	int features: 0
All Features:
	int features: 1
	float features: 863
	object features: 0
Total time taken in fit_transform: 1.6500029563903809
	Data preprocessing and feature engineering runtime = 1.67s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Saving AutogluonModels/ag-20201027_140459\learner.pkl
Saving AutogluonModels/ag-20201027_140459\utils\data\X_train.pkl
Saving AutogluonModels/

In [33]:
end_time = time.time()
print("Time spent", end_time - start_time)

In [35]:
test_data = train.tail(29000)
performance = predictor.evaluate(test_data)

Total time taken in transform: 0.9569988250732422
Loading: AutogluonModels/ag-20201027_140459\models\weighted_ensemble_k0_l1\model.pkl
Loading: AutogluonModels/ag-20201027_140459\models\CatboostClassifier_STACKER_l0\model.pkl
Predictive performance on given dataset: accuracy = 0.5517931034482758


In [36]:
joblib.dump(predictor, f"autogluon_{train_size}.pkl")

['autogluon_500.pkl']